## 套件引入 及 資料前處理

In [1]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Activation, Dense, Embedding, LSTM, Bidirectional
from tensorflow.keras.models import Sequential
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from math import log, sqrt
import pandas as pd
import numpy as np
%matplotlib inline

In [2]:
df = pd.read_csv("./file.csv/file.csv")
df.head()

,Unnamed: 0,tweets,labels
0,0,ChatGPT: Optimizing Language Models for Dialog...,neutral
1,1,"Try talking with ChatGPT, our new AI system wh...",good
2,2,ChatGPT: Optimizing Language Models for Dialog...,neutral
3,3,"THRILLED to share that ChatGPT, our new model ...",good
4,4,"As of 2 minutes ago, @OpenAI released their ne...",bad


### 刪除不必要的欄位

In [3]:
# Remove the 'Unnamed: 0' column

df = df[['tweets', 'labels']]
df.head()

,tweets,labels
0,ChatGPT: Optimizing Language Models for Dialog...,neutral
1,"Try talking with ChatGPT, our new AI system wh...",good
2,ChatGPT: Optimizing Language Models for Dialog...,neutral
3,"THRILLED to share that ChatGPT, our new model ...",good
4,"As of 2 minutes ago, @OpenAI released their ne...",bad


### 刪除文中的 https:// 網址

In [4]:
# Remove all the tweet links since they all begin with https:

df['tweet_list'] = df['tweets'].str.split('https:')
df.head()

,tweets,labels,tweet_list
0,ChatGPT: Optimizing Language Models for Dialog...,neutral,[ChatGPT: Optimizing Language Models for Dialo...
1,"Try talking with ChatGPT, our new AI system wh...",good,"[Try talking with ChatGPT, our new AI system w..."
2,ChatGPT: Optimizing Language Models for Dialog...,neutral,[ChatGPT: Optimizing Language Models for Dialo...
3,"THRILLED to share that ChatGPT, our new model ...",good,"[THRILLED to share that ChatGPT, our new model..."
4,"As of 2 minutes ago, @OpenAI released their ne...",bad,"[As of 2 minutes ago, @OpenAI released their n..."


### 選取需要的欄位

In [5]:
# Select the text part of the list

text = [i[0] for i in df.tweet_list]
df['text'] = text
df = df[['text', 'labels']]
df.head()

,text,labels
0,ChatGPT: Optimizing Language Models for Dialogue,neutral
1,"Try talking with ChatGPT, our new AI system wh...",good
2,ChatGPT: Optimizing Language Models for Dialogue,neutral
3,"THRILLED to share that ChatGPT, our new model ...",good
4,"As of 2 minutes ago, @OpenAI released their ne...",bad


### 修改欄位名稱

In [6]:
# Change the title: 'review' as 'description'; 'sentiment' as 'label'
description = [i for i in df.text]
label = [i for i in df.labels]
df['description'] = description
df['label'] = label
df = df[['description', 'label']]
df.head()

,description,label
0,ChatGPT: Optimizing Language Models for Dialogue,neutral
1,"Try talking with ChatGPT, our new AI system wh...",good
2,ChatGPT: Optimizing Language Models for Dialogue,neutral
3,"THRILLED to share that ChatGPT, our new model ...",good
4,"As of 2 minutes ago, @OpenAI released their ne...",bad


### 將 label 內 'positive' 標註為 '1', 'neutral' 標註為'0', 'negative' 標註為 '-1'

In [7]:
# Mark the 'good' as '1','neutral' as '0', and 'bad' as '-1'
df['label'] = np.where(df['label']=='good', 1, np.where(df['label']=='bad', -1, 0))
df.head()

,description,label
0,ChatGPT: Optimizing Language Models for Dialogue,0
1,"Try talking with ChatGPT, our new AI system wh...",1
2,ChatGPT: Optimizing Language Models for Dialogue,0
3,"THRILLED to share that ChatGPT, our new model ...",1
4,"As of 2 minutes ago, @OpenAI released their ne...",-1


In [8]:
# 分割訓練集和測試集
X_train, X_test, y_train, y_test = train_test_split(df['description'], df['label'], test_size=0.2)

In [9]:
# 將文本轉換為數字序列
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

In [10]:
# 填充序列,使其長度相同
maxlen = 100
X_train_pad = pad_sequences(X_train_seq, maxlen=maxlen)
X_test_pad = pad_sequences(X_test_seq, maxlen=maxlen)

## 定義模型

In [11]:
#定義模型
model = Sequential()
model.add(LSTM(64, input_shape=(maxlen, 1)))
model.add(Dense(1, activation='softmax'))  #應使用 softmax 或直接刪除

## 編譯模型

In [12]:
#編譯模型
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

## 訓練模型

In [13]:
#訓練模型
model.fit(X_train_pad, y_train, validation_data=(X_test_pad, y_test), epochs=15, batch_size=32)

Epoch 1/15
5483/5483 [==============================] - 310s 56ms/step - loss: -38.2478 - accuracy: 0.2558 - val_loss: -75.7445 - val_accuracy: 0.2539
Epoch 2/15
5483/5483 [==============================] - 282s 51ms/step - loss: -110.0037 - accuracy: 0.2558 - val_loss: -148.5142 - val_accuracy: 0.2539
Epoch 3/15
5483/5483 [==============================] - 251s 46ms/step - loss: -182.1761 - accuracy: 0.2558 - val_loss: -221.7513 - val_accuracy: 0.2539
Epoch 4/15
5483/5483 [==============================] - 321s 59ms/step - loss: -253.8418 - accuracy: 0.2558 - val_loss: -293.8283 - val_accuracy: 0.2539
Epoch 5/15
5483/5483 [==============================] - 256s 47ms/step - loss: -325.5043 - accuracy: 0.2558 - val_loss: -367.5163 - val_accuracy: 0.2539
Epoch 6/15
5483/5483 [==============================] - 255s 47ms/step - loss: -387.3332 - accuracy: 0.2558 - val_loss: -429.0525 - val_accuracy: 0.2539
Epoch 7/15
5483/5483 [==============================] - 368s 67ms/step - loss: -457.

## 評估模型

In [14]:
#評估模型
loss, acc = model.evaluate(X_test_pad, y_test, batch_size=32)
print('Test accuracy:', acc)

1371/1371 [==============================] - 23s 17ms/step - loss: -1073.4672 - accuracy: 0.2539
Test accuracy: 0.2538589537143707


## 將測試的語句轉為索引後，預測

In [15]:
x = ['ChatGPT is optimizing language models for dialogue.']
test_sequences = tokenizer.texts_to_sequences(x)

# Pad the testing sequences
test_padded = pad_sequences(test_sequences, maxlen=maxlen)

model.predict(test_padded)

1/1 [==============================] - 1s 940ms/step


array([[1.]], dtype=float32)

In [16]:
x = ['Bad model.']
test_sequences = tokenizer.texts_to_sequences(x)

# Pad the testing sequences
test_padded = pad_sequences(test_sequences, maxlen=maxlen)

model.predict(test_padded)

1/1 [==============================] - 0s 34ms/step


array([[1.]], dtype=float32)

In [17]:
x = ['The ChatGPT model is good today.']
test_sequences = tokenizer.texts_to_sequences(x)

# Pad the testing sequences
test_padded = pad_sequences(test_sequences, maxlen=maxlen)

model.predict(test_padded)

1/1 [==============================] - 0s 31ms/step


array([[1.]], dtype=float32)